In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Endometriosis"
cohort = "GSE145702"

# Input paths
in_trait_dir = "../../input/GEO/Endometriosis"
in_cohort_dir = "../../input/GEO/Endometriosis/GSE145702"

# Output paths
out_data_file = "../../output/preprocess/Endometriosis/GSE145702.csv"
out_gene_data_file = "../../output/preprocess/Endometriosis/gene_data/GSE145702.csv"
out_clinical_data_file = "../../output/preprocess/Endometriosis/clinical_data/GSE145702.csv"
json_path = "../../output/preprocess/Endometriosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Steroid Hormones Regulate Genome-Wide Epigenetic Programming and Gene Transcription in Human Endometrial Cells with Marked Aberrancies in Endometriosis"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['gender: Female'], 1: ['cell type: Eutopic endometrial stromal fibroblasts'], 2: ['disease state: Normal', 'disease state: Endometriosis Stage I', 'disease state: Endometriosis Stage IV'], 3: ['treatment: E2', 'treatment: P4', 'treatment: E2+P4', 'treatment: vehicle'], 4: ['individual: NUP1', 'individual: NUP2', 'individual: NUP3', 'individual: NUP4', 'individual: Endo I-1', 'individual: Endo I-2', 'individual: Endo I-3', 'individual: Endo I-4', 'individual: Endo IV-1', 'individual: Endo IV-2', 'individual: Endo IV-3', 'individual: Endo IV-4']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Callable, Optional, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, this appears to be a SuperSeries with gene transcription data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Trait (Endometriosis): Available in row 2 under "disease state"
trait_row = 2

# Age: Not available in the sample characteristics
age_row = None

# Gender: Available in row 0, but it's constant (all Female)
gender_row = None  # Though gender is available, it's constant so we mark it as unavailable

# 2.2 Data Type Conversion
def convert_trait(value: str) -> int:
    """Convert Endometriosis status to binary: 0 for Normal, 1 for Endometriosis."""
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if "Normal" in value:
        return 0
    elif "Endometriosis" in value:
        return 1
    else:
        return None

def convert_age(value: str) -> float:
    """Convert age to float. Not used since age data is not available."""
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

def convert_gender(value: str) -> int:
    """Convert gender to binary: 0 for Female, 1 for Male. Not used since gender data is constant."""
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    else:
        return None

# 3. Save Metadata
# Trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path, 
                             is_gene_available=is_gene_available, 
                             is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Assuming clinical_data has been loaded in a previous step
    # Create a sample dataframe for testing if clinical_data is not defined
    try:
        clinical_data
    except NameError:
        # Using sample characteristics as a placeholder
        sample_chars = {
            0: ['gender: Female'], 
            1: ['cell type: Eutopic endometrial stromal fibroblasts'], 
            2: ['disease state: Normal', 'disease state: Endometriosis Stage I', 'disease state: Endometriosis Stage IV'], 
            3: ['treatment: E2', 'treatment: P4', 'treatment: E2+P4', 'treatment: vehicle'], 
            4: ['individual: NUP1', 'individual: NUP2', 'individual: NUP3', 'individual: NUP4', 'individual: Endo I-1', 'individual: Endo I-2', 'individual: Endo I-3', 'individual: Endo I-4', 'individual: Endo IV-1', 'individual: Endo IV-2', 'individual: Endo IV-3', 'individual: Endo IV-4']
        }
        
        # Convert to a format similar to a dataframe
        clinical_data = pd.DataFrame()
        for key, values in sample_chars.items():
            clinical_data[key] = values
    
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(clinical_features)
    print("Preview of clinical features:")
    print(preview)
    
    # Save to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical features:
{'GSM4331199': [0.0], 'GSM4331200': [0.0], 'GSM4331201': [0.0], 'GSM4331202': [0.0], 'GSM4331203': [0.0], 'GSM4331204': [0.0], 'GSM4331205': [0.0], 'GSM4331206': [0.0], 'GSM4331207': [0.0], 'GSM4331208': [0.0], 'GSM4331209': [0.0], 'GSM4331210': [0.0], 'GSM4331211': [0.0], 'GSM4331212': [0.0], 'GSM4331213': [0.0], 'GSM4331214': [0.0], 'GSM4331215': [1.0], 'GSM4331216': [1.0], 'GSM4331217': [1.0], 'GSM4331218': [1.0], 'GSM4331219': [1.0], 'GSM4331220': [1.0], 'GSM4331221': [1.0], 'GSM4331222': [1.0], 'GSM4331223': [1.0], 'GSM4331224': [1.0], 'GSM4331225': [1.0], 'GSM4331226': [1.0], 'GSM4331227': [1.0], 'GSM4331228': [1.0], 'GSM4331229': [1.0], 'GSM4331230': [1.0], 'GSM4331231': [1.0], 'GSM4331232': [1.0], 'GSM4331233': [1.0], 'GSM4331234': [1.0], 'GSM4331235': [1.0], 'GSM4331236': [1.0], 'GSM4331237': [1.0], 'GSM4331238': [1.0], 'GSM4331239': [1.0], 'GSM4331240': [1.0], 'GSM4331241': [1.0], 'GSM4331242': [1.0], 'GSM4331243': [1.0], 'GSM4331244': [1.0], 'GS

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 68
Header line: "ID_REF"	"GSM4331199"	"GSM4331200"	"GSM4331201"	"GSM4331202"	"GSM4331203"	"GSM4331204"	"GSM4331205"	"GSM4331206"	"GSM4331207"	"GSM4331208"	"GSM4331209"	"GSM4331210"	"GSM4331211"	"GSM4331212"	"GSM4331213"	"GSM4331214"	"GSM4331215"	"GSM4331216"	"GSM4331217"	"GSM4331218"	"GSM4331219"	"GSM4331220"	"GSM4331221"	"GSM4331222"	"GSM4331223"	"GSM4331224"	"GSM4331225"	"GSM4331226"	"GSM4331227"	"GSM4331228"	"GSM4331229"	"GSM4331230"	"GSM4331231"	"GSM4331232"	"GSM4331233"	"GSM4331234"	"GSM4331235"	"GSM4331236"	"GSM4331237"	"GSM4331238"	"GSM4331239"	"GSM4331240"	"GSM4331241"	"GSM4331242"	"GSM4331243"	"GSM4331244"	"GSM4331245"	"GSM4331246"
First data line: 7892501	5.71675	3.834201	4.159606	5.485716	6.177502	3.268134	5.447216	4.924031	6.030231	5.966432	5.049547	5.640725	5.178561	5.42022	5.878601	5.105815	5.226443	5.879782	5.619013	5.150586	5.291864	4.175157	6.070517	4.793396	6.301941	3.389179	6.073273	4.432051	4.978615	5.589209	5.530389	5.291379	4.393631	4.501

### Step 4: Gene Identifier Review

In [5]:
# Based on examining the gene identifiers in the gene expression data,
# these appear to be probe IDs (numeric IDs like 7892501) from a microarray platform,
# not standard human gene symbols.
# These will need to be mapped to standard gene symbols for analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['7896736', '7896738', '7896740', '7896742', '7896744'], 'GB_LIST': [nan, nan, 'NM_001004195,NM_001005240,NM_001005484,BC136848,BC136867,BC136907,BC136908', 'NR_024437,XM_006711854,XM_006726377,XR_430662,AK298283,AL137655,BC032332,BC118988,BC122537,BC131690,NM_207366,AK301928,BC071667', 'NM_001005221,NM_001005224,NM_001005277,NM_001005504,BC137547,BC137568'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008', 'chr1:334129-334296', 'chr1:367659-368597'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['53049', '63015', '69091', '334129', '367659'], 'RANGE_STOP': ['54936', '63887', '70008', '334296', '368597'], 'total_probes': [7.0, 31.0, 24.0, 6.0, 36.0], 'gene_assignment': ['---', 'ENST00000328113 // OR4G2P // olfactory receptor, family 4, subfamily G, member 2 pseudogene // ---

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on the gene annotation preview, we need to identify the key columns
# "ID" in gene_annotation corresponds to the probe identifiers in gene_data
# "gene_assignment" contains gene symbol information, though it needs extraction

# 2. Create a gene mapping dataframe with ID and gene symbols
# The ID column contains the probe identifiers
# We'll use "gene_assignment" column to extract gene symbols

# Inspect the gene annotation structure to verify these are the correct columns
# Extract probe IDs and corresponding gene symbol information
gene_mapping = get_gene_mapping(gene_annotation, prob_col="ID", gene_col="gene_assignment")

# Preview the mapping dataframe
print("Gene mapping preview (first 5 rows):")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the size of the gene expression dataframe after mapping
print(f"Gene expression data shape after mapping: {gene_data.shape}")

# Preview the first few genes and their expression values
print("Preview of gene expression data (first 5 genes):")
print(gene_data.head())

# Save gene expression data to the output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping preview (first 5 rows):
        ID                                               Gene
0  7896736                                                ---
1  7896738  ENST00000328113 // OR4G2P // olfactory recepto...
2  7896740  NM_001004195 // OR4F4 // olfactory receptor, f...
3  7896742  NR_024437 // LOC728323 // uncharacterized LOC7...
4  7896744  NM_001005221 // OR4F29 // olfactory receptor, ...


Gene expression data shape after mapping: (117474, 48)
Preview of gene expression data (first 5 genes):
      GSM4331199  GSM4331200  GSM4331201  GSM4331202  GSM4331203  GSM4331204  \
Gene                                                                           
A-     27.476003   27.551134   27.838619   28.155517   26.874872   29.123108   
A-3-    0.644641    0.634150    0.695572    0.620117    0.582717    0.682781   
A-52    1.515156    1.507943    1.500627    1.497666    1.526996    1.516796   
A-E     1.044291    0.993303    1.012745    1.055889    1.015062    1.055954   
A-I     1.983325    1.975007    1.967200    1.965546    1.938795    2.039291   

      GSM4331205  GSM4331206  GSM4331207  GSM4331208  ...  GSM4331237  \
Gene                                                  ...               
A-     27.542408   28.407916   27.569581   28.085677  ...   28.049284   
A-3-    0.659926    0.648423    0.634608    0.656936  ...    0.607944   
A-52    1.527090    1.536073    1.544126   

Gene expression data saved to ../../output/preprocess/Endometriosis/gene_data/GSE145702.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Create clinical features directly from clinical_data using the conversion functions defined earlier
clinical_features_df = geo_select_clinical_features(
    clinical_data, 
    trait=trait, 
    trait_row=trait_row, 
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Now link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features_df, normalized_gene_data)
print("Linked data shape:", linked_data.shape)

# Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression from monocytes of rheumatoid arthritis patients, with osteoporosis status included in comorbidity information."
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Normalized gene data saved to ../../output/preprocess/Endometriosis/gene_data/GSE145702.csv
Clinical data saved to ../../output/preprocess/Endometriosis/clinical_data/GSE145702.csv
Linked data shape: (48, 24230)


For the feature 'Endometriosis', the least common label is '0.0' with 16 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'Endometriosis' in this dataset is fine.



Linked data saved to ../../output/preprocess/Endometriosis/GSE145702.csv
